<div align='center'><strong>Wikipedia Pageviews Project</strong>
<br />
<i>Netanel Madmoni</i>
</div>

----------------

# Introduction

In [1]:
#imports
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm import tqdm
from functools import reduce
from itertools import product
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import json
import seaborn as sns
from rich import print

load_dotenv()

RAW_DATA_DIR = Path(os.getenv('RAW_DATA_DIR'))
PROCESSED_DATA_DIR = Path(os.getenv('PROCESSED_DATA_DIR'))

# Raw Data Acquisition

1. Get List of people on Wikipedia
```sql
SELECT DISTINCT en_wiki -- page title name in english wikipedia	
FROM  `rising-theater-416315.wikipedia_pageviews.wikidata`,
      UNNEST(instance_of) AS instance_of_struct

    
WHERE instance_of_struct.numeric_id = 5 -- instance_of = 5 => person
```

2. Get pagevoew data for those people

      ```sql
      SELECT title, DATETIME_TRUNC(datehour, MONTH) AS month, SUM(views) AS monthly_views

                  
      FROM  `rising-theater-416315.wikipedia_pageviews.pageviews_2023` a
            JOIN `rising-theater-416315.data_for_project.distinct_people` b
            ON a.title = b.en_wiki

      
      WHERE datehour IS NOT NULL
      AND wiki = "en"

      GROUP BY title, DATETIME_TRUNC(datehour, MONTH)
      ```

3. Get wikidata for those people
      ```sql
      SELECT *	
      FROM  `rising-theater-416315.wikipedia_pageviews.wikidata`,
      UNNEST(instance_of) AS instance_of_struct

    
      WHERE instance_of_struct.numeric_id = 5
      ```

# Data transformation

## Views Data

Are organized in csv files, one per year.

In [6]:
# Read raw data
dfs = []
for file in RAW_DATA_DIR.glob(r'monthly_views_*.csv'):
    print(f'{file.name} - {os.stat(file).st_size / 1024 ** 2:.2f} MB)')
    dfs.append(pl.read_csv(file))

print(f'Total files: {len(dfs)}')

monthly_views_2015.csv - 411.06 MB)
monthly_views_2016.csv - 658.91 MB)
monthly_views_2019.csv - 803.42 MB)
monthly_views_2020.csv - 851.54 MB)
monthly_views_2017.csv - 708.39 MB)
monthly_views_2022.csv - 949.62 MB)
monthly_views_2023.csv - 980.47 MB)
monthly_views_2021.csv - 904.32 MB)
monthly_views_2018.csv - 755.98 MB)
Total files: 0


In [ ]:
# Pivot
transformed_dfs = []
for df in tqdm(dfs):
    transformed_df = (df.with_columns(pl.col('month')
                                    .map_elements(lambda s: f'{s[:4]}-{s[5:7]}')
                                    .alias('year_month'))
                      .pivot(index='title', columns='year_month', values='monthly_views')
    )
    transformed_dfs.append(transformed_df)

In [ ]:
# Join
all_names = pl.concat([df.select('title') for df in dfs]).unique()
print(f'{len(all_names):,}')
views_df = all_names
for i, df in tqdm(enumerate(transformed_dfs)):
    views_df = df.join(views_df, 'title', 'outer_coalesce')
views_df = views_df.select(['title', *sorted([c for c in views_df.columns if c != 'title'])])
views_df.write_parquet(f'{PROCESSED_DATA_DIR}/pageviews_all.parquet')

views_df

## Entity Data

Are organized in json files.

In [9]:
info_files = (RAW_DATA_DIR / 'wikidata').glob('*')
len(list(info_files))

400

For example...

In [86]:
info_files = (RAW_DATA_DIR / 'wikidata').glob('*')
with open(next(info_files)) as f:
    for line in f:
        d = json.loads(line)
        sites = [sitelink['site'] for sitelink in d['sitelinks']]
        if 'enwiki' in sites:
            print(d)
            break
    

{
    'id': 'Q17122148',
    'numeric_id': '17122148',
    'en_label': 'Albrecht Joseph',
    'en_wiki': 'Albrecht_Joseph',
    'en_description': 'German screenwriter, film editor and author',
    'type': 'item',
    'sitelinks': [
        {'site': 'dewiki', 'title': 'Albrecht Joseph', 'encoded': 'Albrecht_Joseph'},
        {'site': 'enwiki', 'title': 'Albrecht Joseph', 'encoded': 'Albrecht_Joseph'},
        {'site': 'frwiki', 'title': 'Albrecht Joseph', 'encoded': 'Albrecht_Joseph'},
        {'site': 'arwiki', 'title': 'ألبرشت جوزيف', 'encoded': 'ألبرشت_جوزيف'},
        {'site': 'arzwiki', 'title': 'البرشت چوزيف', 'encoded': 'البرشت_چوزيف'},
        {'site': 'fawiki', 'title': 'آلبرشت یوزف', 'encoded': 'آلبرشت_یوزف'}
    ],
    'descriptions': [
        {'language': 'de', 'value': 'deutsch-amerikanischer Theater- und Filmschaffender'},
        {'language': 'fa', 'value': 'فیلمنامه\u200cنویس، تدوینگر، و نویسنده آلمانی'},
        {'language': 'en', 'value': 'German screenwriter, film editor and author'},
        {'language': 'nl', 'value': 'Duits scenarioschrijver (1901-1991)'},
        {'language': 'sv', 'value': 'tysk författare och filmklippare'}
    ],
    'labels': [
        {'language': 'de', 'value': 'Albrecht Joseph'},
        {'language': 'fr', 'value': 'Albrecht Joseph'},
        {'language': 'nl', 'value': 'Albrecht Joseph'},
        {'language': 'en', 'value': 'Albrecht Joseph'},
        {'language': 'es', 'value': 'Albrecht Joseph'},
        {'language': 'hu', 'value': 'Albrecht Joseph'},
        {'language': 'sl', 'value': 'Albrecht Joseph'},
        {'language': 'ca', 'value': 'Albrecht Joseph'},
        {'language': 'ast', 'value': 'Albrecht Joseph'},
        {'language': 'ar', 'value': 'ألبرشت جوزيف'},
        {'language': 'cs', 'value': 'Albrecht Joseph'},
        {'language': 'sk', 'value': 'Albrecht Joseph'},
        {'language': 'arz', 'value': 'البرشت چوزيف'},
        {'language': 'it', 'value': 'Albrecht Joseph'},
        {'language': 'sq', 'value': 'Albrecht Joseph'},
        {'language': 'he', 'value': "אלברכט ג'וזף"},
        {'language': 'sv', 'value': 'Albrecht Joseph'},
        {'language': 'ga', 'value': 'Albrecht Joseph'},
        {'language': 'fa', 'value': 'آلبرشت یوزف'},
        {'language': 'ru', 'value': 'Элбрехт Джозеф'}
    ],
    'aliases': [
        {'language': 'de', 'value': 'Al Joseph'},
        {'language': 'en', 'value': 'Al Joseph'},
        {'language': 'cs', 'value': 'Al Joseph'},
        {'language': 'sk', 'value': 'Al Joseph'}
    ],
    'instance_of': [{'numeric_id': '5'}],
    'gender': [{'numeric_id': '6581097'}],
    'date_of_birth': [{'time': '+1901-11-20T00:00:00Z'}],
    'date_of_death': [{'time': '+1901-11-20T00:00:00Z'}],
    'worked_at': [],
    'country_of_citizenship': [{'numeric_id': '183'}],
    'country': [],
    'educated_at': [],
    'occupation': [{'numeric_id': '28389'}, {'numeric_id': '7042855'}, {'numeric_id': '36180'}],
    'instrument': [],
    'genre': [],
    'industry': [],
    'subclass_of': [],
    'coordinate_location': [],
    'iso_3166_alpha3': [],
    'member_of': [],
    'from_fictional_universe': []
}

In [2]:
try:
    info_df = pl.read_parquet(PROCESSED_DATA_DIR / 'wikidata_all.parquet')
except FileNotFoundError:
    info_list = []
    info_files = (RAW_DATA_DIR / 'wikidata').glob('*')
    for file in info_files:
        with open(file) as f:
            for line in f:
                d = json.loads(line)
                sites = [sitelink['site'] for sitelink in d['sitelinks']]
                if 'enwiki' in sites:
                    info_list.append(d)
    info_df = pl.DataFrame(info_list)
    
info_df.head()

id,numeric_id,en_label,en_wiki,en_description,type,sitelinks,descriptions,labels,aliases,instance_of,gender,date_of_birth,date_of_death,worked_at,country_of_citizenship,country,educated_at,occupation,instrument,genre,industry,subclass_of,coordinate_location,iso_3166_alpha3,member_of,from_fictional_universe
str,str,str,str,str,str,list[struct[3]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[2]],list[null],list[struct[1]],list[struct[1]]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""item""","[{""dewiki"",""Albrecht Joseph"",""Albrecht_Joseph""}, {""enwiki"",""Albrecht Joseph"",""Albrecht_Joseph""}, … {""fawiki"",""آلبرشت یوزف"",""آلبرشت_یوزف""}]","[{""de"",""deutsch-amerikanischer Theater- und Filmschaffender""}, {""fa"",""فیلمنامه‌نویس، تدوینگر، و نویسنده آلمانی""}, … {""sv"",""tysk författare och filmklippare""}]","[{""de"",""Albrecht Joseph""}, {""fr"",""Albrecht Joseph""}, … {""ru"",""Элбрехт Джозеф""}]","[{""de"",""Al Joseph""}, {""en"",""Al Joseph""}, … {""sk"",""Al Joseph""}]","[{""5""}]","[{""6581097""}]","[{""+1901-11-20T00:00:00Z""}]","[{""+1901-11-20T00:00:00Z""}]",[],"[{""183""}]",[],[],"[{""28389""}, {""7042855""}, {""36180""}]",[],[],[],[],[],[],[],[]
"""Q100707809""","""100707809""","""Robert Cahaly""","""Robert_Cahaly""","""American polls…","""item""","[{""enwiki"",""Robert Cahaly"",""Robert_Cahaly""}, {""jawiki"",""ロバート・カヘリー"",""ロバート・カヘリー""}]","[{""en"",""American pollster and political consultant""}, {""zh"",""美国民调专家""}]","[{""en"",""Robert Cahaly""}, {""de"",""Robert Cahaly""}, … {""pt-br"",""Robert Cahaly""}]",[],"[{""5""}]","[{""6581097""}]",[],[],"[{""100704905""}]","[{""30""}]",[],"[{""1024426""}]","[{""16919156""}, {""8125919""}]",[],[],[],[],[],[],[],[]
"""Q8988407""","""8988407""","""Li Fei""","""Lei_Fei""","""actor, busines…","""item""","[{""zhwiki"",""李菲"",""李菲""}, {""enwiki"",""Lei Fei"",""Lei_Fei""}]","[{""nl"",""ondernemer""}, {""en"",""actor, businesswoman, and wushu taolu athlete from Macau""}]","[{""zh"",""李菲""}, {""en"",""Li Fei""}, … {""sq"",""Jewel Lee""}]","[{""en"",""Jewel Lee""}]","[{""5""}]","[{""6581072""}]",[],[],[],[],[],[],"[{""43845""}]",[],[],[],[],[],[],[],[]
"""Q3588237""","""3588237""","""Yang Ti-liang""","""Yang_Ti-liang""","""Hong Kong judg…","""item""","[{""enwiki"",""Yang Ti-liang"",""Yang_Ti-liang""}, {""zhwiki"",""楊鐵樑"",""楊鐵樑""}, … {""kowiki"",""양톄량"",""양톄량""}]","[{""en"",""Hong Kong judge (1929–2023)""}, {""nl"",""rechter uit Brunei""}, … {""ko"",""홍콩의 판사 (1929–2023)""}]","[{""en"",""Yang Ti-liang""}, {""en-ca"",""Yang Ti-liang""}, … {""pap"",""Yang Ti-liang""}]","[{""en"",""Sir Ti-liang Yang""}, {""ko"",""양티량""}, {""ko"",""양철량""}]","[{""5""}]","[{""6581097""}]","[{""+1929-06-30T00:00:00Z""}]","[{""+1929-06-30T00:00:00Z""}]",[],"[{""148""}]",[],"[{""1639978""}, {""193196""}]","[{""16533""}, {""329455""}, {""82955""}]",[],[],[],[],[],[],"[{""157412""}]",[]
"""Q521785""","""521785""","""Mauro Sérgio V…","""Maurinho_(foot…","""Brazilian foot…","""item""","[{""itwiki"",""Mauro Sérgio Viriato Mendes"",""Mauro_Sérgio_Viriato_Mendes""}, {""plwiki"",""Mauro Sérgio Viriato Mendes"",""Mauro_Sérgio_Viriato_Mendes""}, … {""trwiki"",""Maurinho (1978 doğumlu futbolcu)"",""Maurinho_(1978_doğumlu_futbolcu)""}]","[{""it"",""calciatore brasiliano""}, {""fr"",""joueur de football brésilien""}, … {""mos"",""Bal tãongra""}]","[{""it"",""Mauro Sérgio Viriato Mendes""}, {""en"",""Mauro Sérgio Viriato Mendes""}, … {""tr"",""Maurinho (1978 doğumlu futbolcu)""}]","[{""es"",""Mauro Sergio Viriato Mendes""}, {""es"",""Maurinho""}, … {""fr"",""Maurinho""}]","[{""5""}]","[{""6581097""}]","[{""+1978-10-11T00:00:00Z""}]","[{""+1978-10-11T00:00:00Z""}]",[],"[{""155""}]",[],[],"[{""937857""}]",[],[],[],[],[],[],[],[]


Clean info dataframe

In [3]:
print(dict(zip(info_df.columns, info_df.dtypes)))

{
    'id': String,
    'numeric_id': String,
    'en_label': String,
    'en_wiki': String,
    'en_description': String,
    'type': String,
    'sitelinks': List(Struct({'site': String, 'title': String, 'encoded': String})),
    'descriptions': List(Struct({'language': String, 'value': String})),
    'labels': List(Struct({'language': String, 'value': String})),
    'aliases': List(Struct({'language': String, 'value': String})),
    'instance_of': List(Struct({'numeric_id': String})),
    'gender': List(Struct({'numeric_id': String})),
    'date_of_birth': List(Struct({'time': String})),
    'date_of_death': List(Struct({'time': String})),
    'worked_at': List(Struct({'numeric_id': String})),
    'country_of_citizenship': List(Struct({'numeric_id': String})),
    'country': List(Struct({'numeric_id': String})),
    'educated_at': List(Struct({'numeric_id': String})),
    'occupation': List(Struct({'numeric_id': String})),
    'instrument': List(Struct({'numeric_id': String})),
    'genre': List(Struct({'numeric_id': String})),
    'industry': List(Struct({'numeric_id': String})),
    'subclass_of': List(Struct({'numeric_id': String})),
    'coordinate_location': List(Struct({'latitude': Float64, 'longitude': Float64})),
    'iso_3166_alpha3': List(Null),
    'member_of': List(Struct({'numeric_id': String})),
    'from_fictional_universe': List(Struct({'numeric_id': String}))
}

**Questions for cleaning the data**
1. Can `en_wiki` be a primary key instead of the id columns, in order to match this table to the pageviews one?
1. Are there any columns that contain nothing but empty lists or null values? If so - drop them.
1. Are there any list/struct columns that always contain a single value? If so - flatten them
1. How many unique values does the column `type` contain? If 1 - redundant.
1. Can an entity only have one gender? If so - convert from list of structs to a simple string (gender code). Also, can an entity have no gender? If so - why?

In [51]:
print(info_df['en_wiki'].n_unique(), info_df.height)

1993447 1993459

In [53]:
info_df.filter(info_df['en_wiki'].is_duplicated()).sort('en_wiki')

id,numeric_id,en_label,en_wiki,en_description,type,sitelinks,descriptions,labels,aliases,instance_of,gender,date_of_birth,date_of_death,worked_at,country_of_citizenship,country,educated_at,occupation,instrument,genre,industry,subclass_of,coordinate_location,iso_3166_alpha3,member_of,from_fictional_universe
str,str,str,str,str,str,list[struct[3]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[2]],list[null],list[struct[1]],list[struct[1]]
"""Q125017311""","""125017311""","""Ashutosh""","""Ashutosh_(spir…",null,"""item""","[{""enwiki"",""Ashutosh (spiritual leader)"",""Ashutosh_(spiritual_leader)""}]",[],"[{""en"",""Ashutosh""}]",[],"[{""5""}]","[{""6581097""}]","[{""+1946-01-01T00:00:00Z""}]","[{""+1946-01-01T00:00:00Z""}]",[],[],[],[],[],[],[],[],[],[],[],[],[]
"""Q17488903""","""17488903""","""Ashutosh""","""Ashutosh_(spir…","""Indian spiritu…","""item""","[{""enwiki"",""Ashutosh (spiritual leader)"",""Ashutosh_(spiritual_leader)""}, {""hiwiki"",""आशुतोष (आध्यात्मिक नेता)"",""आशुतोष_(आध्यात्मिक_नेता)""}]","[{""en"",""Indian spiritual leader""}, {""hi"",""भारतीय आध्यात्मिक गुरु""}]","[{""en"",""Ashutosh""}, {""fr"",""Ashutosh""}, … {""hi"",""आशुतोष (आध्यात्मिक नेता)""}]","[{""en"",""Ashutosh Maharaj""}, {""en"",""Mahesh Kumar Jha""}]","[{""5""}]","[{""6581097""}]","[{""+1946-01-01T00:00:00Z""}]","[{""+1946-01-01T00:00:00Z""}]",[],"[{""668""}, {""129286""}, {""1775277""}]",[],[],"[{""64711780""}, {""484260""}]",[],[],[],[],[],[],[],[]
"""Q99237902""","""99237902""","""Ferdinand van …","""Ferdinand_van_…","""Italian noblem…","""item""","[{""enwiki"",""Ferdinand van den Eynde, 1st Marquess of Castelnuovo"",""Ferdinand_van_den_Eynde,_1st_Marquess_of_Castelnuovo""}, {""nlwiki"",""Ferdinand van den Eynde"",""Ferdinand_van_den_Eynde""}, … {""frwiki"",""Ferdinand van den Eynde (1er marquis de Castelnuovo)"",""Ferdinand_van_den_Eynde_(1er_marquis_de_Castelnuovo)""}]","[{""en"",""Italian nobleman and magnate of Flemish origin (d.1674)""}, {""fr"",""noble italien""}, {""it"",""nobile, mercante, collezionista d'arte e mecenate italiano""}]","[{""en"",""Ferdinand van den Eynde, 1st Marquess of Castelnuovo""}, {""nl"",""Ferdinand van den Eynde, 1st Marquess of Castelnuovo""}, … {""fr"",""Ferdinand van den Eynde""}]","[{""en"",""Ferdinand van den Eynde""}, {""en"",""Ferdinando Vandeneynden""}, {""nl"",""Ferdinand van den Eynde""}]","[{""5""}, {""5""}]","[{""6581097""}]",[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
"""Q99237902""","""99237902""","""Ferdinand van …","""Ferdinand_van_…","""Italian noblem…","""item""","[{""enwiki"",""Ferdinand van den Eynde, 1st Marquess of Castelnuovo"",""Ferdinand_van_den_Eynde,_1st_Marquess_of_Castelnuovo""}, {""nlwiki"",""Ferdinand van den Eynde"",""Ferdinand_van_den_Eynde""}, … {""frwiki"",""Ferdinand van den Eynde (1er marquis de Castelnuovo)"",""Ferdinand_van_den_Eynde_(1er_marquis_de_Castelnuovo)""}]","[{""en"",""Italian nobleman and magnate of Flemish origin (d.1674)""}, {""fr"",""noble italien""}, {""it"",""nobile, mercante, collezionista d'arte e mecenate italiano""}]","[{""en"",""Ferdinand van den Eynde, 1st Marquess of Castelnuovo""}, {""nl"",""Ferdinand van den Eynde, 1st Marquess of Castelnuovo""}, … {""fr"",""Ferdinand van den Eynde""}]","[{""en"",""Ferdinand van den Eynde""}, {""en"",""Ferdinando Vandeneynden""}, {""nl"",""Ferdinand van den Eynde""}]","[{""5""}, {""5""}]","[{""6581097""}]",[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
"""Q5638516""","""5638516""","""Hagai Shaham""","""Hagai_Shaham""","""Israeli violin…","""item""","[{""enwiki"",""Hagai Shaham"",""Hagai_Shaham""}, {""jawiki"",""ハガイ・シャハム"",""ハガイ・シャハム""}, … {""arzwiki"",""حجاى شاحام"",""حجاى_شاحام""}]","[{""cs"",""izraelský houslista""}, {""en"",""Israeli violinist""}, … {""fi"",""israelilainen viulisti""}]","[{""en

In [24]:
# For each list column, view distribution of list lengths 
list_columns = [c for c in info_df.columns if info_df[c].dtype == pl.List]
for column in list_columns:
    print(column, info_df[column].map_elements(len).rename(f'length of {column}').value_counts())

sitelinks shape: (260, 2)
┌─────────────────────┬────────┐
│ length of sitelinks ┆ count  │
│ ---                 ┆ ---    │
│ i64                 ┆ u32    │
╞═════════════════════╪════════╡
│ 269                 ┆ 1      │
│ 130                 ┆ 11     │
│ 235                 ┆ 2      │
│ 11                  ┆ 29059  │
│ 5                   ┆ 140116 │
│ …                   ┆ …      │
│ 68                  ┆ 170    │
│ 293                 ┆ 2      │
│ 54                  ┆ 411    │
│ 91                  ┆ 51     │
│ 256                 ┆ 3      │
└─────────────────────┴────────┘

descriptions shape: (131, 2)
┌────────────────────────┬────────┐
│ length of descriptions ┆ count  │
│ ---                    ┆ ---    │
│ i64                    ┆ u32    │
╞════════════════════════╪════════╡
│ 4                      ┆ 100204 │
│ 32                     ┆ 14819  │
│ 43                     ┆ 2297   │
│ 46                     ┆ 1184   │
│ 18                     ┆ 56795  │
│ …                      ┆ …      │
│ 3                      ┆ 111702 │
│ 94                     ┆ 5      │
│ 33                     ┆ 11452  │
│ 82                     ┆ 10     │
│ 38                     ┆ 4174   │
└────────────────────────┴────────┘

labels shape: (290, 2)
┌──────────────────┬───────┐
│ length of labels ┆ count │
│ ---              ┆ ---   │
│ i64              ┆ u32   │
╞══════════════════╪═══════╡
│ 216              ┆ 105   │
│ 225              ┆ 90    │
│ 236              ┆ 28    │
│ 156              ┆ 60    │
│ 93               ┆ 762   │
│ …                ┆ …     │
│ 74               ┆ 1113  │
│ 228              ┆ 53    │
│ 152              ┆ 97    │
│ 210              ┆ 4     │
│ 260              ┆ 17    │
└──────────────────┴───────┘

aliases shape: (412, 2)
┌───────────────────┬───────┐
│ length of aliases ┆ count │
│ ---               ┆ ---   │
│ i64               ┆ u32   │
╞═══════════════════╪═══════╡
│ 118               ┆ 25    │
│ 153               ┆ 22    │
│ 177               ┆ 7     │
│ 399               ┆ 3     │
│ 375               ┆ 1     │
│ …                 ┆ …     │
│ 69                ┆ 104   │
│ 47                ┆ 379   │
│ 211               ┆ 8     │
│ 645               ┆ 3     │
│ 302               ┆ 1     │
└───────────────────┴───────┘

instance_of shape: (6, 2)
┌───────────────────────┬─────────┐
│ length of instance_of ┆ count   │
│ ---                   ┆ ---     │
│ i64                   ┆ u32     │
╞═══════════════════════╪═════════╡
│ 5                     ┆ 2       │
│ 11                    ┆ 1       │
│ 3                     ┆ 81      │
│ 2                     ┆ 1118    │
│ 1                     ┆ 1992244 │
│ 4                     ┆ 13      │
└───────────────────────┴─────────┘

...

subclass_of shape: (3, 2)
┌───────────────────────┬─────────┐
│ length of subclass_of ┆ count   │
│ ---                   ┆ ---     │
│ i64                   ┆ u32     │
╞═══════════════════════╪═════════╡
│ 0                     ┆ 1993442 │
│ 2                     ┆ 1       │
│ 1                     ┆ 16      │
└───────────────────────┴─────────┘

coordinate_location shape: (3, 2)
┌───────────────────────────────┬─────────┐
│ length of coordinate_location ┆ count   │
│ ---                           ┆ ---     │
│ i64                           ┆ u32     │
╞═══════════════════════════════╪═════════╡
│ 1                             ┆ 24      │
│ 2                             ┆ 1       │
│ 0                             ┆ 1993434 │
└───────────────────────────────┴─────────┘

iso_3166_alpha3 shape: (1, 2)
┌───────────────────────────┬─────────┐
│ length of iso_3166_alpha3 ┆ count   │
│ ---                       ┆ ---     │
│ i64                       ┆ u32     │
╞═══════════════════════════╪═════════╡
│ 0                         ┆ 1993459 │
└───────────────────────────┴─────────┘

member_of shape: (26, 2)
┌─────────────────────┬───────┐
│ length of member_of ┆ count │
│ ---                 ┆ ---   │
│ i64                 ┆ u32   │
╞═════════════════════╪═══════╡
│ 13                  ┆ 48    │
│ 20                  ┆ 3     │
│ 3                   ┆ 7629  │
│ 7                   ┆ 518   │
│ 5                   ┆ 1689  │
│ …                   ┆ …     │
│ 31                  ┆ 1     │
│ 22                  ┆ 3     │
│ 8                   ┆ 331   │
│ 21                  ┆ 4     │
│ 4                   ┆ 3423  │
└─────────────────────┴───────┘

from_fictional_universe shape: (2, 2)
┌───────────────────────────────────┬─────────┐
│ length of from_fictional_univers… ┆ count   │
│ ---                               ┆ ---     │
│ i64                               ┆ u32     │
╞═══════════════════════════════════╪═════════╡
│ 0                                 ┆ 1993454 │
│ 1                                 ┆ 5       │
└───────────────────────────────────┴─────────┘

In [6]:
info_df['type'].n_unique()

1

In [16]:
info_df['gender'].map_elements(len).value_counts()

gender,count
i64,u32
3,15
0,302
1,1992615
2,524
4,3


**Observations:**
There are a few types of columns:
1. Redundant columns: columns that don't add information, either because they contain the same value throughout, have many missing values or do not contribute to our specific needs. The columns are:
    - `type` (same value for all rows)
    - `numeric_id` (contained in `id`)
    - `descriptions` (unneeded information)
    - `labels` (unneeded information)
    - `coordinate_location`(>99% empty)
    - `country` (according to the Wikidata site, this property should not be used for humans)
    - `date_of_death` (has wrong values)
These columns will be dropped.
2. Columns with a single value per row: columns that (should) have zero or one values for each row. These columns are:
    - `id`
    - `en_label`
    - `en_wiki`
    - `en_description`
    - `date of birth`
These columns will be... 
3. Columns with a few *'interesting'* values per row: columns that contain zero, one or more values for each row, where every value is *interesting* for our purpose. These columns are:
    - `instance_of`
    - `gender` (apparently an entity can have multiple genders)
4. Columns that have multiple values per row: columns that have multiple values per row, where only an aggregation of these values is *interesting* for our purposes. These columns are:
    - `sitelinks`
    - `aliases`
------

    - `instrument`
    - `worked_at`
    - `country_of_citizenship`
    - `educated_at`
    - `occupation`
    - `genre`
    - `industry`
    - `subclass_of`
    - `member_of`
    - `from_fictional_universe`


- `sitelinks` contains the name of this page in all relevant wikimedia sites
    - Convert this to binary columns: `exists_in_{sitename}`
    - Also create an aggregated column: `num_sitelinks`
- `descriptions`, `labels`
    - I will drop these columns
- `aliases`
    - I will convert this to numeric columns: `num_aliases_in_{sitename}`
- `instance_of`
    -  will convert this to binary columns: `is_instance_of_{parent}`
- `gender` - not always a single one.
    - I will convert this to binary columns: `has_gender_{gender}`
- `date of birth`, `date of death`:
    - Convert to timestamps
- All one-dimensional struct columns:
    - flatten
- `iso_3166_alpha3`
    - drop


In [43]:
def unnest_list_of_structs(l: list[pl.Struct]):
    '''
    Turn a list of structs into a list of the first value of each struct.
    For example:
    `[{'a': 1, 'b': 2}, {'a': 3, 'b': 4}] --> [1, 3]`
    '''
    first_key = l[0].fields[0].name
    return []

info_df['sitelinks'].list.eval(pl.element().struct[0])

sitelinks
list[str]
"[""dewiki"", ""enwiki"", … ""fawiki""]"
"[""enwiki"", ""jawiki""]"
"[""zhwiki"", ""enwiki""]"
"[""enwiki"", ""zhwiki"", … ""kowiki""]"
"[""itwiki"", ""plwiki"", … ""trwiki""]"
…
"[""enwiki"", ""nowiki"", … ""nlwiki""]"
"[""frwiki"", ""kowiki"", … ""anpwiki""]"
"[""frwiki"", ""fawiki"", … ""ruwiki""]"


In [46]:
def clean_info_df(df: pl.DataFrame):
    columns_to_drop = ['type', 'descriptions', 'labels', 'iso_3166_alpha3', 'coordinate_location']
    struct_columns = [c for c in df.columns if df[c].dtype == pl.List and c not in columns_to_drop]
    columns_to_encode = [c for c in df.columns if df[c].dtype == pl.List and c not in
                         [*columns_to_drop, 'date_of_birth', 'date_of_death']]
    
    df = (df
          .drop(columns_to_drop) # drop columns
          .with_columns([pl.col(col).list.eval(pl.element().struct[0]) for col in struct_columns]) # unnest structs
    )
    
    unique_sitelinks = df['sitelinks'].explode().unique()
    unique_parents = df['instance_of'].explode().unique()
    unique_
    
    sitelinks_columns = [pl.col('sitelinks').list.contains(site).alias(f'is_in_{site}') for site in unique_sitelinks]
    parents_columns = [pl.col('instance_of').list.contains(parent).alias(f'is_instance_of_{parent}') for parent in unique_parents]
    
    
    df = (df
          .with_columns([
           *sitelinks_columns,
           pl.col('aliases').list.len().alias('num_aliases'),
           *parents_columns,
           
          ]))
    
    return df

clean_info_df(info_df.head(5)).explode('sitelinks').pivot(index='id', )

id,numeric_id,en_label,en_wiki,en_description,sitelinks,aliases,instance_of,gender,date_of_birth,date_of_death,worked_at,country_of_citizenship,country,educated_at,occupation,instrument,genre,industry,subclass_of,coordinate_location,member_of,from_fictional_universe
str,str,str,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[f64],list[str],list[str]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""dewiki""","[""de"", ""en"", … ""sk""]","[""5""]","[""6581097""]","[""+1901-11-20T00:00:00Z""]","[""+1901-11-20T00:00:00Z""]",[],"[""183""]",[],[],"[""28389"", ""7042855"", ""36180""]",[],[],[],[],[],[],[]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""enwiki""","[""de"", ""en"", … ""sk""]","[""5""]","[""6581097""]","[""+1901-11-20T00:00:00Z""]","[""+1901-11-20T00:00:00Z""]",[],"[""183""]",[],[],"[""28389"", ""7042855"", ""36180""]",[],[],[],[],[],[],[]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""frwiki""","[""de"", ""en"", … ""sk""]","[""5""]","[""6581097""]","[""+1901-11-20T00:00:00Z""]","[""+1901-11-20T00:00:00Z""]",[],"[""183""]",[],[],"[""28389"", ""7042855"", ""36180""]",[],[],[],[],[],[],[]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""arwiki""","[""de"", ""en"", … ""sk""]","[""5""]","[""6581097""]","[""+1901-11-20T00:00:00Z""]","[""+1901-11-20T00:00:00Z""]",[],"[""183""]",[],[],"[""28389"", ""7042855"", ""36180""]",[],[],[],[],[],[],[]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""arzwiki""","[""de"", ""en"", … ""sk""]","[""5""]","[""6581097""]","[""+1901-11-20T00:00:00Z""]","[""+1901-11-20T00:00:00Z""]",[],"[""183""]",[],[],"[""28389"", ""7042855"", ""36180""]",[],[],[],[],[],[],[]
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q521785""","""521785""","""Mauro Sérgio V…","""Maurinho_(foot…","""Brazilian foot…","""enwiki""","[""es"", ""es"", … ""fr""]","[""5""]","[""6581097""]","[""+1978-10-11T00:00:00Z""]","[""+1978-10-11T00:00:00Z""]",[],"[""155""]",[],[],"[""937857""]",[],[],[],[],[],[],[]
"""Q521785""","""521785""","""Mauro Sérgio V…","""Maurinho_(foot…","""Brazilian foot…","""arzwiki""","[""es"", ""es"", … ""fr""]","[""5""]","[""6581097""]","[""+1978-10-11T00:00:00Z""]","[""+1978-10-11T00:00:00Z""]",[],"[""155""]",[],[],"[""937857""]",[],[],[],[],[],[],[]
"""Q521785""","""521785""","""Mauro Sérgio V…","""Maurinho_(foot…","""Brazilian foot…","""ptwiki""","[""es"", ""es"", … ""fr""]","[""5""]","[""6581097""]","[""+1978-10-11T00:00:00Z""]","[""+1978-10-11T00:00:00Z""]",[],"[""155""]",[],[],"[""937857""]",[],[],[],[],[],[],[]


In [96]:
info_df.write_parquet(PROCESSED_DATA_DIR / 'wikidata_all.parquet')

Aliases not included in the pageviews data

Convert code to label description:
- https://stackoverflow.com/questions/59737076/how-to-get-a-label-of-a-property-from-wikidata
- https://stackoverflow.com/questions/72704205/how-to-convert-wiki-data-qid-to-entity-and-vice-versa-in-python

In [72]:
with pl.Config(tbl_rows=500):
    display(info_df.filter(pl.col('gender').list.len() > 1).transpose(include_header=True))

column,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_505,column_506,column_507,column_508,column_509,column_510,column_511,column_512,column_513,column_514,column_515,column_516,column_517,column_518,column_519,column_520,column_521,column_522,column_523,column_524,column_525,column_526,column_527,column_528,column_529,column_530,column_531,column_532,column_533,column_534,column_535,column_536,column_537,column_538,column_539,column_540,column_541
str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
"""id""","[""Q7800135""]","[""Q108381040""]","[""Q55397010""]","[""Q5214843""]","[""Q2994979""]","[""Q107472755""]","[""Q4859279""]","[""Q8051479""]","[""Q1294046""]","[""Q18686095""]","[""Q7116433""]","[""Q55834726""]","[""Q20035643""]","[""Q106785836""]","[""Q7857048""]","[""Q85199656""]","[""Q8023575""]","[""Q3492524""]","[""Q89372815""]","[""Q11467933""]","[""Q708445""]","[""Q55731552""]","[""Q62129117""]","[""Q16300043""]","[""Q100994887""]","[""Q124218234""]","[""Q6256128""]","[""Q1347851""]","[""Q18518779""]","[""Q21062531""]","[""Q114632978""]","[""Q96694248""]","[""Q11548763""]","[""Q106095215""]","[""Q41173""]","[""Q89377845""]",…,"[""Q117552132""]","[""Q6653592""]","[""Q20528796""]","[""Q16731062""]","[""Q20712237""]","[""Q59171056""]","[""Q788298""]","[""Q55314957""]","[""Q29908714""]","[""Q65042596""]","[""Q107118557""]","[""Q275025""]","[""Q706927""]","[""Q83689574""]","[""Q1281867""]","[""Q7802962""]","[""Q117425346""]","[""Q19367437""]","[""Q107118477""]","[""Q704375""]","[""Q20508909""]","[""Q53951730""]","[""Q84429493""]","[""Q27244414""]","[""Q68631381""]","[""Q24700929""]","[""Q6574985""]","[""Q20683861""]","[""Q84196087""]","[""Q13129290""]","[""Q7098066""]","[""Q1033604""]","[""Q1368840""]","[""Q715407""]","[""Q97930223""]","[""Q21062831""]","[""Q42291398""]"
"""numeric_id""","[""7800135""]","[""108381040""]","[""55397010""]","[""5214843""]","[""2994979""]","[""107472755""]","[""4859279""]","[""8051479""]","[""1294046""]","[""18686095""]","[""7116433""]","[""55834726""]","[""20035643""]","[""106785836""]","[""7857048""]","[""85199656""]","[""8023575""]","[""3492524""]","[""89372815""]","[""11467933""]","[""708445""]","[""55731552""]","[""62129117""]","[""16300043""]","[""100994887""]","[""124218234""]","[""6256128""]","[""1347851""]","[""18518779""]","[""21062531""]","[""114632978""]","[""96694248""]","[""11548763""]","[""106095215""]","[""41173""]","[""89377845""]",…,"[""117552132""]","[""6653592""]","[""20528796""]","[""16731062""]","[""20712237""]","[""59171056""]","[""788298""]","[""55314957""]","[""29908714""]","[""65042596""]","[""107118557""]","[""275025""]","[""706927""]","[""83689574""]","[""1281867""]","[""7802962""]","[""117425346""]","[""19367437""]","[""107118477""]","[""704375""]","[""20508909""]","[""53951730""]","[""84429493""]","[""27244414""]","[""68631381""]","[""24700929""]","[""6574985""]","[""20683861""]","[""84196087""]","[""13129290""]","[""7098066""]","[""1033604""]","[""1368840""]","[""715407""]","[""979302

In [60]:
(info_df['numeric_id'] == info_df['id'].str.replace('Q', '')).all()

True

# Time Series Clustering

In [ ]:
views_df = pl.read_parquet(PROCESSED_DATA_DIR / 'pageviews_all.parquet')

In [98]:
views_df[:30]

title,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,…,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Neophyte_II_of…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,14,17,26,10,17,24,29,32,51,20,36,46,32,119,27,32,18,26,22,30,30,26,35,33,33,26,25,24,29,44,23,25
"""Olivia_Giovett…",26,34,52,20,41,40,19,21,27,23,32,21,32,29,57,47,31,21,24,15,42,17,45,30,25,34,36,22,22,22,16,28,30,34,22,19,…,39,30,32,34,37,29,14,26,24,36,39,35,20,63,33,39,19,19,30,20,14,21,35,52,25,25,33,20,18,22,31,26,16,20,59,41,19
"""Alexander_Gran…",9,48,24,23,32,24,12,20,19,7,8,14,18,27,11,23,12,19,15,17,11,13,26,14,16,18,13,20,13,26,16,19,32,14,18,12,…,5,13,9,12,11,9,14,19,9,7,5,11,4,11,9,7,9,9,4,5,12,72,6,10,10,10,2,7,10,7,2,7,4,9,10,1,5
"""Bernard_H._Rae…",17,16,21,20,19,18,10,9,17,5,8,10,5,28,19,11,17,14,13,19,16,13,24,18,10,10,14,13,14,19,5,15,26,12,13,10,…,2,1,4,1,5,5,3,5,2,2,3,4,null,12,4,3,2,1,4,null,1,3,8,6,3,1,5,1,3,5,7,3,1,2,null,1,2
"""Leigh_Magar""",47,46,64,36,56,45,41,26,30,48,15,26,19,38,28,38,37,51,24,36,17,15,45,39,26,22,21,14,19,23,16,32,36,21,18,20,…,8,8,9,13,6,3,12,13,12,5,8,8,11,3,9,8,13,13,9,20,9,8,14,27,14,13,20,13,10,9,5,9,9,8,5,10,2
"""Carolyn_Stanfo…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,221,294,359,286,650,202,170,169,83,97,107,112,70,78,88,64,54,78,84,74,131,74,86,180,105,75,76,73,105,56,50,89,86,59,58,50,42
"""Boncho_Novakov…",16,18,26,19,21,16,11,14,18,6,6,15,4,10,10,15,14,12,11,18,14,4,22,13,15,16,8,13,11,8,7,14,18,9,7,11,…,4,7,5,3,18,39,5,3,5,5,2,4,5,3,5,5,4,6,7,3,4,5,5,5,3,9,2,3,4,4,3,7,9,2,3,2,3
"""Margaret_Hodge…",5665,2932,2420,2482,3097,2693,1955,1867,3365,2059,1808,3435,4174,12893,4331,4311,3434,2573,2110,1829,1813,1953,1667,2523,2442,3777,1705,1577,5159,1650,3699,1623,2057,1368,2277,2749,…,2260,1819,2069,1605,1630,1890,2082,1391,1126,1351,2217,1430,4400,1788,2612,1680,1471,1743,1336,2097,1272,1918,2196,1194,1204,1536,2234,1909,1311,1261,3170,2097,1026,1335,5767,2327,3310
"""Carmen_Belén_R…",null,null,null,null,null,25,26,31,27,60,35,32,29,20,20,51,23,31,37,31,59,46,51,35,28,31,34,34,25,42,27,39,118,56,36,46,…,17,24,56,80,25,32,31,13,24,25,18,27,27,18,46,54,20,24,30,12,14,12,25,27,10,31,36,63,29,12,10,15,36,20,15,20,17


In [118]:
X_train = views_df.select([col for col in views_df.columns if col != 'title']).fill_null(0).to_numpy()
X_train

array([[ 0,  0,  0, ..., 44, 23, 25],
       [26, 34, 52, ..., 59, 41, 19],
       [ 9, 48, 24, ..., 10,  1,  5],
       ...,
       [40, 29, 34, ...,  3,  1,  2],
       [23, 52, 38, ...,  8,  7,  6],
       [ 0,  0,  0, ..., 16,  8,  7]])

In [119]:
X_train.shape

(1976658, 104)

In [120]:
import numpy
import matplotlib.pyplot as plt

from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler

seed = 0
numpy.random.seed(seed)

sz = X_train.shape[1]

X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)

# Euclidean k-means
print("Euclidean k-means")
km = TimeSeriesKMeans(n_clusters=3, verbose=True, random_state=seed,
n_jobs=-1)
y_pred = km.fit_predict(X_train)

plt.figure()
for yi in range(3):
    plt.subplot(3, 3, yi + 1)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Euclidean $k$-means")

# DBA-k-means
print("DBA k-means")
dba_km = TimeSeriesKMeans(n_clusters=3,
                          n_init=2,
                          metric="dtw",
                          verbose=True,
                          max_iter_barycenter=10,
                          random_state=seed,
                          n_jobs=-1)
y_pred = dba_km.fit_predict(X_train)

for yi in range(3):
    plt.subplot(3, 3, 4 + yi)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(dba_km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("DBA $k$-means")


plt.tight_layout()
plt.show()

Euclidean k-means

143.854 --> 76.592 --> 

KeyboardInterrupt: 

In [112]:
km.inertia_

70.60124743054607

In [114]:
y_pred

array([2, 1, 1, 0, 0, 2, 0, 1, 2, 1, 2, 2, 0, 0, 2, 0, 0, 1, 1, 2, 0, 1,
       2, 2, 0, 0, 1, 0, 0, 2, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 2, 1, 0, 2, 0, 2, 0, 1, 2,
       2, 2, 0, 0, 1, 1, 2, 2, 1, 1, 0, 1, 1, 0, 2, 0, 2, 0, 2, 1, 2, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 2, 2])

In [115]:
y = views_df[:100].with_columns(pl.Series(name='cluster', values=y_pred))

https://github.com/aeon-toolkit/aeon/blob/4a58c4dba5bb37e43979368ad9cf092540e71786/examples/segmentation/segmentation_with_clasp.ipynb